In [ ]:
import os
import xml.etree.ElementTree as ET

# fpath = os.path.join(self.ann_dir, index + '.xml')
tree = ET.parse('/Users/ashwinvaswani/Documents/vlr_hw1/q1_q2_classification/data/VOCdevkit/VOC2007/Annotations/000001.xml')

In [ ]:
root = tree.getroot()
for elem in root:
    # print("1 ", elem.tag, elem.attrib, elem.text, elem.tail)
    # print("2 ",elem)
    if elem.tag == "object":
        print("3 ", elem[0].text, elem[3].text)
    # print()

In [ ]:
# initialize our implementation of ResNet
# import tqdm
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import torchvision
import utils
from utils import ARGS
import torch

args = ARGS(
        epochs=50,
        inp_size=64,
        # use_cuda=True,
        use_cuda=False,
        val_every=70,
        lr=0.001,
        batch_size=64,
        step_size=30,
        gamma=0.1
    )

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 20)
model.eval()
device = 'cpu'

test_loader = utils.get_data_loader(
        'voc', train=False, batch_size=args.test_batch_size, split='test', inp_size=args.inp_size)
 
labels = []
# outputs = np.array([])
features = None
# for batch in tqdm(test_loader, desc='Running the model inference'):
for batch_idx, (data, target, wgt) in enumerate(test_loader):
    images = data.to(device)
    # labels += batch['label']
    labels.extend(target)
    # image_paths += batch['image_path']
 
    output = model(images)
    print(output.shape)
    current_outputs = output.detach().cpu().numpy()
    if features is None:
        features = current_outputs
    else:
        features = np.concatenate((features, current_outputs))
    if batch_idx == 1:
        break

tsne = TSNE(n_components=2).fit_transform(features)

# scale and move the coordinates so they fit [0; 1] range
def scale_to_01_range(x):
    # compute the distribution range
    value_range = (np.max(x) - np.min(x))
 
    # move the distribution so that it starts from zero
    # by extracting the minimal value from all its values
    starts_from_zero = x - np.min(x)
 
    # make the distribution fit [0; 1] by dividing by its range
    return starts_from_zero / value_range


In [ ]:
# extract x and y coordinates representing the positions of the images on T-SNE plot
tx = tsne[:, 0]
ty = tsne[:, 1]
 
tx = scale_to_01_range(tx)
ty = scale_to_01_range(ty)

# initialize a matplotlib plot
fig = plt.figure()
ax = fig.add_subplot(111)
 
colors_per_class = {
        0 : [254, 202, 87],
    1 : [255, 107, 107],
    2 : [10, 189, 227],
    3 : [255, 159, 243],
    4 : [16, 172, 132],
    5 : [128, 80, 128],
    6 : [87, 101, 116],
    7 : [52, 31, 151],
    8 : [0, 0, 0],
    9 : [100, 100, 255],
    10: [12, 253, 251],
    11: [204, 69, 253],
    12: [191, 141, 125],
    13: [252, 100, 170],
    14: [18, 42, 176],
    15: [23, 109, 77],
    16: [46, 60, 220],
    17: [95, 108, 232],
    18: [255, 0, 0],
    19: [0, 255, 0],
    # 20: [0, 0, 255]
}

# for every class, we'll add a scatter plot separately
for label in colors_per_class:

    # find the samples of the current class in the data
    indices = [i for i, l in enumerate(labels) if np.argmax(l) == label]
 
    # extract the coordinates of the points of this class only
    current_tx = np.take(tx, indices)
    current_ty = np.take(ty, indices)
 
    # convert the class color to matplotlib format
    color = np.array(colors_per_class[label], dtype=np.float) / 255
 
    # add a scatter plot with the corresponding color and label
    ax.scatter(current_tx, current_ty, color=color, label=label)
 
# build a legend using the labels we set previously
ax.legend(loc='best')
 
# finally, show the plot
plt.show()